In [1]:
# Copyright (c) 2019-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

In [2]:
#
# Code to generate sentence representations from a pretrained model.
# This can be used to initialize a cross-lingual classifier, for instance.
#

In [1]:
import os
import torch

from src.utils import AttrDict
from src.data.dictionary import Dictionary, BOS_WORD, EOS_WORD, PAD_WORD, UNK_WORD, MASK_WORD
from src.model.transformer import TransformerModel

FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.


## Reload a pretrained model

In [7]:
# model_path = 'models/mlm_100_1280.pth'
# model_path = 'mlm_tlm_xnli15_1024.pth'
model_path = 'mlm_enfr_1024.pth'

reloaded = torch.load(model_path)
params = AttrDict(reloaded['params'])
print("Supported languages: %s" % ", ".join(params.lang2id.keys()))

Supported languages: ar, bg, de, el, en, es, fr, hi, ru, sw, th, tr, ur, vi, zh


In [9]:
reloaded.keys()

dict_keys(['model', 'dico_id2word', 'dico_word2id', 'dico_counts', 'params'])

In [10]:
reloaded['params']

{'same_enc_dec': True,
 'emb_dim': 1024,
 'n_layers': 12,
 'n_heads': 8,
 'dropout': 0.1,
 'attention_dropout': 0.1,
 'gelu_activation': True,
 'share_inout_emb': True,
 'sinusoidal_embeddings': False,
 'asm': False,
 'max_vocab': 95000,
 'min_count': 0,
 'id2lang': {0: 'ar',
  1: 'bg',
  2: 'de',
  3: 'el',
  4: 'en',
  5: 'es',
  6: 'fr',
  7: 'hi',
  8: 'ru',
  9: 'sw',
  10: 'th',
  11: 'tr',
  12: 'ur',
  13: 'vi',
  14: 'zh'},
 'lang2id': {'ar': 0,
  'bg': 1,
  'de': 2,
  'el': 3,
  'en': 4,
  'es': 5,
  'fr': 6,
  'hi': 7,
  'ru': 8,
  'sw': 9,
  'th': 10,
  'tr': 11,
  'ur': 12,
  'vi': 13,
  'zh': 14},
 'n_langs': 15,
 'n_words': 95000}

## Build dictionary / update parameters / build model

In [6]:
# build dictionary / update parameters
dico = Dictionary(reloaded['dico_id2word'], reloaded['dico_word2id'], reloaded['dico_counts'])
params.n_words = len(dico)
params.bos_index = dico.index(BOS_WORD)
params.eos_index = dico.index(EOS_WORD)
params.pad_index = dico.index(PAD_WORD)
params.unk_index = dico.index(UNK_WORD)
params.mask_index = dico.index(MASK_WORD)

In [ ]:
# build model / reload weights
model = TransformerModel(params, dico, True, True)
model.eval()
model.load_state_dict(reloaded['model'])

In [7]:
from src.model import build_model

params.encoder_only = False
params.reload_emb = ''
params.reload_model = '{0},{0}'.format(model_path)
params.local_rank = -1

en, de = build_model(params, dico)

Parameter layer_norm15.0.weight not found.
Parameter layer_norm15.0.bias not found.
Parameter encoder_attn.0.q_lin.weight not found.
Parameter encoder_attn.0.q_lin.bias not found.
Parameter encoder_attn.0.k_lin.weight not found.
Parameter encoder_attn.0.k_lin.bias not found.
Parameter encoder_attn.0.v_lin.weight not found.
Parameter encoder_attn.0.v_lin.bias not found.
Parameter encoder_attn.0.out_lin.weight not found.
Parameter encoder_attn.0.out_lin.bias not found.
Parameter layer_norm15.1.weight not found.
Parameter layer_norm15.1.bias not found.
Parameter encoder_attn.1.q_lin.weight not found.
Parameter encoder_attn.1.q_lin.bias not found.
Parameter encoder_attn.1.k_lin.weight not found.
Parameter encoder_attn.1.k_lin.bias not found.
Parameter encoder_attn.1.v_lin.weight not found.
Parameter encoder_attn.1.v_lin.bias not found.
Parameter encoder_attn.1.out_lin.weight not found.
Parameter encoder_attn.1.out_lin.bias not found.
Parameter layer_norm15.2.weight not found.
Parameter lay


## Get sentence representations

Sentences have to be in the BPE format, i.e. tokenized sentences on which you applied fastBPE.

Below you can see an example for English, French, Spanish, German, Arabic and Chinese sentences.

In [24]:
# Below is one way to bpe-ize sentences
codes = "codes_enfr" # path to the codes of the model
# codes = ""
fastbpe = os.path.join(os.getcwd(), 'tools/fastBPE/fast')

def to_bpe(sentences):
    # write sentences to tmp file
    with open('/tmp/sentences', 'w') as fwrite:
        for sent in sentences:
            fwrite.write(sent + '\n')
    
    # apply bpe to tmp file
    os.system('%s applybpe /tmp/sentences.bpe /tmp/sentences %s' % (fastbpe, codes))
    
    # load bpe-ized sentences
    sentences_bpe = []
    with open('/tmp/sentences.bpe') as f:
        for line in f:
            sentences_bpe.append(line.rstrip())
    
    return sentences_bpe


In [25]:
sentences = ['dancing once he had worn trendy italian leather shoes and jeans from paris that had cost three hundred euros .']
print(to_bpe(sentences))

['dancing once he had worn trendy itali@@ an leather shoes and jeans from paris that had cost three hundred euros .']


In [27]:
# Below are already BPE-ized sentences

# list of (sentences, lang)
sentences = [
    'once he had worn trendy italian leather shoes and jeans from paris that had cost three hundred euros .', # en
    #'Le français est la seule langue étrangère proposée dans le système éducatif .', # fr
    #'El cadmio produce efectos tóxicos en los organismos vivos , aun en concentraciones muy pequeñas .', # es
    #'Nach dem Zweiten Weltkrieg verbreitete sich Bonsai als Hobby in der ganzen Welt .', # de
    #'وقد فاز في الانتخابات في الجولة الثانية من التصويت من قبل سيدي ولد الشيخ عبد الله ، مع أحمد ولد داداه في المرتبة الثانية .', # ar
    '羅伯特 · 皮爾 斯 生於 1863年 , 在 英國 曼徹斯特 學習 而 成為 一 位 工程師 . 1933年 , 皮爾斯 在 直布羅陀去世 .', # zh
]

# bpe-ize sentences
sentences = to_bpe(sentences)
print('\n\n'.join(sentences))

# check how many tokens are OOV
n_w = len([w for w in ' '.join(sentences).split()])
n_oov = len([w for w in ' '.join(sentences).split() if w not in dico.word2id])
print('Number of out-of-vocab words: %s/%s' % (n_oov, n_w))

# add </s> sentence delimiters
sentences = [(('</s> %s </s>' % sent.strip()).split()) for sent in sentences]

once he had worn trendy itali@@ an leather shoes and jeans from paris that had cost three hundred euros .

羅@@ 伯@@ 特 · 皮@@ 爾 斯 生@@ 於 18@@ 63@@ 年 , 在 英@@ 國 曼@@ 徹@@ 斯@@ 特 學@@ 習 而 成@@ 為 一 位 工@@ 程@@ 師 . 19@@ 33@@ 年 , 皮@@ 爾@@ 斯 在 直@@ 布@@ 羅@@ 陀@@ 去@@ 世 .
Number of out-of-vocab words: 22/66


In [18]:
sentences = ['once he had worn trendy italian leather shoes and jeans from paris that had cost three hundred euros .']
sentences = to_bpe(sentences)
[w for w in ' '.join(sentences).split() if w not in dico.word2id]

['italian']

### Create batch

In [7]:
bs = len(sentences)
slen = max([len(sent) for sent in sentences])

word_ids = torch.LongTensor(slen, bs).fill_(params.pad_index)
for i in range(len(sentences)):
    sent = torch.LongTensor([dico.index(w) for w in sentences[i]])
    word_ids[:len(sent), i] = sent

lengths = torch.LongTensor([len(sent) for sent in sentences])
                             
# NOTE: No more language id (removed it in a later version)
# langs = torch.LongTensor([params.lang2id[lang] for _, lang in sentences]).unsqueeze(0).expand(slen, bs) if params.n_langs > 1 else None
langs = None


### Forward

In [8]:
tensor = model('fwd', x=word_ids, lengths=lengths, langs=langs, causal=False).contiguous()
print(tensor.size())

torch.Size([25, 2, 1024])


The variable `tensor` is of shape `(sequence_length, batch_size, model_dimension)`.

`tensor[0]` is a tensor of shape `(batch_size, model_dimension)` that corresponds to the first hidden state of the last layer of each sentence.

This is this vector that we use to finetune on the GLUE and XNLI tasks.